# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-t4hsl751
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-t4hsl751
  Resolved https://github.com/huggingface/accelerate.git to commit 61bcdaa45d35a77478decc78ce204b54dc108812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.12.0.dev0-py3-none-any.whl size=375796 sha256=d739c7644093feeb3eea3e6b885dcb3336691244311d6230a983ac44ba389329
  Stored in directory: /tmp/pip-ephem-wheel-cache-9tzbzx2g/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

Imports libraries for working with Hugging Face models and PyTorch

- `AutoModelForCausalLM` → loads pre-trained causal language models (predict next token)

- `AutoTokenizer` → converts text ↔ tokens

- `BitsAndBytesConfig` → settings for 8-bit/quantized models (memory-efficient)

- `torch` → PyTorch library (tensors, GPU support)

- `accelerate` → simplifies multi-GPU/mixed-precision training

Causal → predicts next token based on previous tokens (like GPT), unlike encoder models that see the whole input at once.

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

- `Quantization`: Reduces the precision of model weights (e.g., from 16/32-bit → 8/4-bit) to save GPU memory and speed up inference.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,                    # load model in 4-bit weights (small, memory-efficient)
  bnb_4bit_use_double_quant=True,       # apply double quantization for better accuracy
  bnb_4bit_quant_type="nf4",            # type of 4-bit quantization (normal float4)
  bnb_4bit_compute_dtype=torch.bfloat16 # computation precision for operations
)


### ⚡ `BitsAndBytesConfig` quantization class and similar classes:

In the Hugging Face ecosystem and PyTorch there are several similar configuration options/classes for quantization, depending on the library and approach

---

#### **1. Hugging Face / Transformers**

- **`BitsAndBytesConfig`** : Full control over 4-bit / 8-bit quantization:
    - `load_in_4bit`, `bnb_4bit_use_double_quant`, `bnb_4bit_quant_type`, `bnb_4bit_compute_dtype`  
  - **Use:** Large LLMs on limited GPU memory.
- **`load_in_8bit` / `load_in_4bit`** : Shorthand flags for memory-efficient model loading.
- **`bnb_4bit_*` / `bnb_8bit_*` options** : Fine-tune memory vs accuracy trade-offs.

---

#### **2. PEFT + QLoRA**

- **`LoraConfig` / `PeftConfig`**  
  - Parameter-efficient fine-tuning configurations.
  - Often combined with quantized models to save memory during training.

---

#### **3. PyTorch Native Quantization**

- **`torch.quantization.QuantStub` / `DeQuantStub`** : Mark inputs/outputs for quantization.
- **`torch.quantization.QConfig`** : Defines weight & activation quantization (`qint8`, `quint8`, `float16`, `bfloat16`).
- **`torch.quantization.prepare` / `convert`** : Apply quantization transformations to regular PyTorch models.

---

#### **4. Other Libraries**

- **NNCF (OpenVINO / Intel)** : Quantization-aware training & post-training quantization.
- **Intel Neural Compressor**  
  - YAML-based configs for quantization & mixed precision.
  - Production-ready optimization pipelines.

---

##### **Summary**

- **LLMs (Hugging Face):** Use `BitsAndBytesConfig`.  
- **Regular PyTorch models:** Use `torch.quantization` tools.  
- **Production optimization:** NNCF or Intel Neural Compressor.  
- **Trade-off:** Memory savings vs slight potential accuracy loss.


`BitsAndBytesConfig`
- Pros: Huge memory savings, can load massive models on modest GPUs, faster inference.

- Cons: Reduced precision can affect accuracy slightly, some operations may be unstable, and hardware support matters for compute dtype.

To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,           # load model
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### After execution, `foundation_model` is a loaded PyTorch model object of type `AutoModelForCausalLM`.

- It contains the model architecture and quantized weights (from `bnb_config`).

- It is ready to generate text using methods like `.generate()`.

- It is allocated on your GPU(s) or CPU depending on `device_map='auto'`.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)       # loads the tokenizer
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]  # gets the token ID for the string "```" to use as an **end-of-sequence** marker


tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

An **end-of-sequence (EOS) marker** is a special token that tells a language model when to stop generating text.

- Without it, the model might keep producing tokens indefinitely.

- Common EOS markers are special tokens like ``, [EOS], or </s>.

- In my code:
`eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]` The model will stop generating when it outputs the token ID corresponding to ```.

This function wraps the call to *model.generate*

In [8]:
# This function generates text from a causal language model.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

- I can later decode these IDs with `tokenizer.decode()` to get text.

- It uses beam search for better output quality and respects the EOS marker.

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [9]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [15]:
sp_nl2sql = sp_nl2sql.format(question="YOUR QUERY HERE")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `YOUR QUERY HERE`:
    ```sql3
    


In [16]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')        # Tokenizes the prompt(sp_nl2sql) into model-readable token IDs. + Moves them to GPU (cuda) for inference.
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400) # Calls the model to generate SQL tokens based on the prompt.
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)              # Decodes token IDs back into human-readable text (the SQL query).

In [22]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [23]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")
# Formats the output:
# Extracts only the SQL query from the model’s response:
#   1. Splits by the ```sql3 marker
#   2. Removes ending ```
#   3. Takes only the first statement before ;
#   4. Strips whitespace and adds a final ;

SELECT COUNT(*) AS number_of_moons FROM solar_system;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [24]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    `{question}`:
    ```sql3
    """


In [25]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    `Return The name of the best paid employee`:
    ```sql3
    


In [26]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [27]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [28]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [29]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return The name of the best paid employee`:
    ```sql3
    


In [30]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [31]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


Comparing with the previous one:

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


- Oh! The same response!

#Now the question in spanish.


In [32]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Δωσε μου το ονομα του υπαλληλου με τον υψηλοτερο μισθο")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    YOUR TABLES HERE

    ### Samples

    YOUR SAMPLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Δωσε μου το ονομα του υπαλληλου με τον υψηλοτερο μισθο`:
    ```sql3
    


In [33]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [34]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, employees.salary
     FROM employees
     ORDER BY employees.salary DESC NULLS LAST
     LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [36]:
# test sofia
sp_sofia = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [35]:
# test sofia
sp_nl2sql_sofia = sp_sofia.format(question="How many moons do we have in our solar system?")
print(sp_nl2sql_sofia)

input_sentences = tokenizer(sp_nl2sql_sofia, return_tensors="pt").to('cuda')        # Tokenizes the prompt(sp_nl2sql) into model-readable token IDs. + Moves them to GPU (cuda) for inference.
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400) # Calls the model to generate SQL tokens based on the prompt.
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `How many moons do we have in our solar system?`:
    ```sql3
    


In [37]:
torch.cuda.empty_cache()
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT COUNT(*) AS number_of_moons FROM solar_system;
